In [1]:
import sys
sys.path.append('../../../src')

import numpy as np
import pandas as pd
from sklearn import svm
from p1.models.utils import get_data_col, get_project_data, get_attr_nums
from p1.models.utils import creat_pred_file, calc_acc, normalize

%load_ext autoreload
%autoreload 2

## Data Loading

In [2]:
# Get Files
data_dir = "../project_data/data"
labels = ["guilty", "not guilty"]

(_, _, tfidf, misc), Y = get_project_data(opts=[False, False, True, True])

## Experiments

In [3]:
tfidf_eval, tfidf_test, tfidf_train = (tfidf[i] for i in range(3))
misc_train, misc_eval, misc_test = (misc[i] for i in range(3))
Y_train, Y_eval, Y_test = (Y[i] for i in range(3))

In [4]:
del tfidf, misc, Y

In [35]:
X_train = np.column_stack([
    tfidf_train.to_numpy(),
    get_attr_nums(misc_train, attr_id="defendant_age", uk="not known", unknown='-1')[:,np.newaxis],
    # get_data_col(misc_train, "defendant_gender"),
    get_data_col(misc_train, "num_victims")
])
X_eval = np.column_stack([
    tfidf_eval.to_numpy(),
    get_attr_nums(misc_eval, attr_id="defendant_age", uk="not known", unknown='-1')[:, np.newaxis],
    # get_data_col(misc_eval, "defendant_gender"),
    get_data_col(misc_eval, "num_victims")
])
X_test = np.column_stack([
    tfidf_test.to_numpy(),
    get_attr_nums(misc_test, attr_id="defendant_age", uk="not known", unknown='-1')[:, np.newaxis],
    # get_data_col(misc_test, "defendant_gender"),
    get_data_col(misc_test, "num_victims")
])

In [36]:
mask = (X_train[:, -2] >= 18) & (X_train[:, -2] != -1); X_train[mask] = 1.0 # known adults
mask = (X_train[:, -2] < 18) & (X_train[:, -2] != -1); X_train[mask] = 1.0 # known children
# X_train[X_train == 'male'] = 1.0; X_train[X_train == 'female'] = 0.0; X_train[X_train == 'indeterminate'] = -1
# X_train[:, 2] = X_train[:, 2]/X_train[:, 2].max(); X_train = X_train.astype(np.float64)
# X_train[:, -1] = normalize(X_train[:, -1]); X_train = X_train.astype(np.float64)
# X_train[:, -2] = normalize(X_train[:, -2])
X_train = normalize(X_train)
# X_train[:, -1] = normalize(X_train[:, -1])

mask = (X_eval[:, -2] >= 18) & (X_eval[:, -2] != -1); X_eval[mask] = 1.0 # known adults
mask = (X_eval[:, -2] < 18) & (X_eval[:, -2] != -1); X_eval[mask] = 1.0 # known children
# X_eval[X_eval == 'male'] = 1.0; X_eval[X_eval == 'female'] = 0.0; X_eval[X_eval == 'indeterminate'] = -1
# X_eval[:, 2] = X_eval[:, 2]/X_eval[:, 2].max(); X_eval = X_eval.astype(np.float64)
# X_eval[:, -1] = normalize(X_eval[:, -1]); X_eval = X_eval.astype(np.float64)
# X_eval[:, -2] = normalize(X_eval[:, -2])
X_eval = normalize(X_eval)
# X_eval[:, -1] = normalize(X_eval[:, -1])


mask = (X_test[:, -2] >= 18) & (X_test[:, -2] != -1); X_test[mask] = 1.0 # known adults
mask = (X_test[:, -2] < 18) & (X_test[:, -2] != -1); X_test[mask] = 1.0 # known children
# X_test[X_test == 'male'] = 1.0; X_test[X_test == 'female'] = 0.0; X_test[X_test == 'indeterminate'] = -1
# X_test[:, 2] = X_test[:, 2]/X_test[:,2].max(); X_test = X_test.astype(np.float64)
# X_test[:, -1] = normalize(X_test[:, -1]); X_test = X_test.astype(np.float64)
# X_test[:, -2] = normalize(X_test[:, -2])
X_test = normalize(X_test)
# X_test[:, -1] = normalize(X_test[:, -1])

### SVM Training

In [42]:
# Train model
model = svm.LinearSVC(tol=1e-10, max_iter=int(1e3), C=0.5, dual='auto')
model.fit(X_train, Y_train.astype(np.float32))

# print test accuracy
print(calc_acc(model.predict(X_test), Y_test))

0.8146666666666667


In [8]:
# Save pred file
creat_pred_file(model.predict(X_eval[:,-2:]), filename="final_svm_scikit_misc_JL.csv")